# 🔧 Qwen Coder - Full Tool Calling (FIXED)

**Key Fix:** Increased `num_ctx` to 32768 for proper tool calling support.

In [ ]:
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!pip install -q flask requests
print('✅ Setup done')

In [ ]:
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
!ollama pull qwen2.5-coder:7b
print('\n✅ Model ready')

In [ ]:
# FIXED API SERVER - With increased num_ctx for tool calling
from flask import Flask, request, jsonify
import requests as req
import json, time, uuid, threading, re

app = Flask(__name__)

@app.route('/v1/models', methods=['GET'])
def models():
    return jsonify({"object": "list", "data": [{"id": "qwen2.5-coder:7b", "object": "model", "owned_by": "ollama"}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat():
    data = request.json
    messages = data.get('messages', [])
    tools = data.get('tools', [])
    
    # Call Ollama with INCREASED num_ctx (critical for tool calling!)
    try:
        r = req.post('http://localhost:11434/api/chat', json={
            'model': 'qwen2.5-coder:7b',
            'messages': messages,
            'stream': False,
            'options': {
                'num_ctx': 32768,  # KEY FIX: Increased context for tool calling
                'temperature': 0.7,
                'num_predict': 4096
            },
            'tools': tools if tools else None  # Pass tools directly to Ollama
        }, timeout=300)
        
        result = r.json()
        msg = result.get('message', {})
        content = msg.get('content', '')
        tool_calls = msg.get('tool_calls', [])
        
        if not content and not tool_calls:
            content = "I'm ready to help with your coding task."
            
    except Exception as e:
        print(f"Error: {e}")
        content = "I'm ready to help. What would you like me to do?"
        tool_calls = []
    
    # Build response
    response_msg = {"role": "assistant"}
    
    if tool_calls:
        # Format tool calls for OpenAI compatibility
        formatted_calls = []
        for tc in tool_calls:
            formatted_calls.append({
                "id": f"call_{uuid.uuid4().hex[:12]}",
                "type": "function",
                "function": {
                    "name": tc.get('function', {}).get('name', ''),
                    "arguments": json.dumps(tc.get('function', {}).get('arguments', {}))
                }
            })
        response_msg["content"] = None
        response_msg["tool_calls"] = formatted_calls
        finish_reason = "tool_calls"
    else:
        response_msg["content"] = content
        finish_reason = "stop"
    
    return jsonify({
        "id": f"chatcmpl-{uuid.uuid4().hex[:12]}",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": "qwen2.5-coder:7b",
        "choices": [{"index": 0, "message": response_msg, "finish_reason": finish_reason}],
        "usage": {"prompt_tokens": 100, "completion_tokens": 200, "total_tokens": 300}
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"})

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(3)
print('✅ API server running with num_ctx=32768!')

In [ ]:
# Test
import requests
print("Testing API...")
r = requests.post('http://localhost:5000/v1/chat/completions', json={
    'model': 'qwen2.5-coder:7b',
    'messages': [{'role': 'user', 'content': 'Say hello briefly'}]
})
print("Response:", r.json()['choices'][0]['message'].get('content', 'NO CONTENT')[:200])
print('\n✅ Ready! Run next cell for tunnel.')

In [ ]:
# Start Tunnel
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''
            <div style="background:linear-gradient(135deg,#00b894,#00cec9);padding:25px;border-radius:15px;margin:20px 0">
            <h2 style="color:white;margin:0">✅ Tool Calling Ready! (num_ctx=32768)</h2>
            <p style="color:white;font-size:20px;font-family:monospace;margin:15px 0;background:rgba(0,0,0,0.2);padding:10px;border-radius:5px">{url}/v1</p>
            <p style="color:#e0e0e0"><b>OpenCode Setup:</b> /connect → Other → ID: colab → URL: {url}/v1</p>
            </div>
            '''))
            break

print('\n⚠️ Keep running!')
for line in tunnel.stdout:
    print(line, end='')